In [4]:
import findspark
findspark.init()

from pprint import pprint

from pyspark.sql import SparkSession
from pyspark.sql.functions import col


spark = SparkSession.builder \
    .master("local") \
    .appName("SparkApp") \
    .getOrCreate()

In [8]:
df = spark.read.format('csv') \
    .option('header',True) \
    .load('data.csv')

print(df.count())

df.show()

22
+-----------+------------+------------+
|   username|airport_code|time_visited|
+-----------+------------+------------+
|alphabeta_1|         ATL|  1629254806|
|alphabeta_1|         ATL|  1629254807|
|alphabeta_1|         ATL|  1629254808|
|alphabeta_1|         ORD|  1629254801|
|alphabeta_1|         ORD|  1629254802|
|alphabeta_1|         ORD|  1629254803|
|alphabeta_1|         ORD|  1629254804|
|alphabeta_1|         CDG|  1629254806|
|alphabeta_2|         ATL|  1629254806|
|alphabeta_2|         ATL|  1629254807|
|alphabeta_2|         ATL|  1629254808|
|alphabeta_2|         ORD|  1629254801|
|alphabeta_2|         ORD|  1629254802|
|alphabeta_2|         ORD|  1629254803|
|alphabeta_2|         ORD|  1629254804|
|alphabeta_2|         CDG|  1629254806|
|alphabeta_2|         DUS|  1629254816|
|alphabeta_2|         DUS|  1629254817|
|alphabeta_2|         HGH|  1629254804|
|alphabeta_2|         HGH|  1629254806|
+-----------+------------+------------+
only showing top 20 rows



In [9]:
data = []

unique_usernames = df.dropDuplicates(['username'])
for row in unique_usernames.collect():
    current_username = row['username']
    print("Current Username: ", current_username)
    
    df1 = df.filter(col('username') == current_username).groupBy("airport_code").count().orderBy("count").select("*").limit(5)
    
    print("Airport code & Number of visits")
    df1.show()
    
    print("Iterating unique airport codes to fetch latest time_visited")
    for row1 in df1.collect():
        df2 = df.filter((col('username') == current_username) & (col('airport_code') == row1['airport_code'])).orderBy("time_visited", ascending=False).select("*").limit(1).collect()
        data.append([current_username, df2[0]['airport_code'], df2[0]['time_visited']])

print("Complete Data Ouput")
pprint(data)

Current Username:  alphabeta_1
Airport code & Number of visits


+------------+-----+
|airport_code|count|
+------------+-----+
|         CDG|    1|
|         ATL|    3|
|         ORD|    4|
+------------+-----+

Iterating unique airport codes to fetch latest time_visited


Current Username:  alphabeta_2
Airport code & Number of visits


+------------+-----+
|airport_code|count|
+------------+-----+
|         CDG|    1|
|         DUS|    2|
|         ATL|    3|
|         ORD|    4|
|         HGH|    4|
+------------+-----+

Iterating unique airport codes to fetch latest time_visited


Complete Data Ouput
[['alphabeta_1', 'CDG', '1629254806'],
 ['alphabeta_1', 'ATL', '1629254808'],
 ['alphabeta_1', 'ORD', '1629254804'],
 ['alphabeta_2', 'CDG', '1629254806'],
 ['alphabeta_2', 'DUS', '1629254817'],
 ['alphabeta_2', 'ATL', '1629254808'],
 ['alphabeta_2', 'ORD', '1629254804'],
 ['alphabeta_2', 'HGH', '1629254817']]
